## Magentic-One
https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/magentic-one.html

In [ ]:
%pip install "autogen-ext[file-surfer]==0.4.0.dev13"

## Load Azure Configuration

In [10]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [11]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)


# Helper Method to see the responses per agent

In [12]:
def get_agent_responses(response):
    # ANSI escape code for bold text
    bold_start = "\033[1m"
    bold_end = "\033[0m"

    # ANSI escape code for red text
    red_start = "\033[31m"
    red_end = "\033[0m"

    print(f"{bold_start}****************Agent Responses****************{bold_end}")

    for messages in response.messages:
        source = messages.source
        print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
        print(messages.content)

## Magentic-One with Assistant Agent

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console


assistant = AssistantAgent(
    "Assistant",
    model_client=az_model_client,
)

team = MagenticOneGroupChat([assistant], model_client=az_model_client)
response = await Console(team.run_stream(task="Provide a different proof for Fermat's Last Theorem"))
get_agent_responses(response)


## Magentic-One with a MultimodalWebSurfer

In [ ]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=az_model_client,
    )

team = MagenticOneGroupChat([surfer], model_client=az_model_client)
response = await Console(team.run_stream(task="Is there a Lakers game on January 5, 2025?"))
get_agent_responses(response)

## Magentic-One with a Code Executor Agent

In [ ]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

# Create a code executor agent that uses a Docker container to execute code.
code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
await code_executor.start()
code_executor_agent = CodeExecutorAgent("code_executor", code_executor=code_executor)

team = MagenticOneGroupChat([code_executor_agent], model_client=az_model_client, max_turns=50)
task = f"""
You have been provided with a dataset named bigfootsightings.csv

What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar

"""
response = await Console(team.run_stream(task=task))

get_agent_responses(response)

# Stop the code executor.
await code_executor.stop()

## Magentic-One with Multiple Agents

In [ ]:
from autogen_ext.agents.file_surfer._file_surfer import FileSurfer
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

# Create a hiking assistant agent.
hiking_assistant = AssistantAgent(
    "HikingAssistant",
    description="You will create a hiking plan based on a location given to you",
    model_client=az_model_client,
    system_message="You will create a hiking plan based on a location given to you"
)

file_path = "../Data/products/"
# Create a file surfer agent.
file_surfer = FileSurfer(
    "FileSurfer",
    model_client=az_model_client
)
file_surfer._browser.open_path(file_path)

# Create a web surfer agent.
surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=az_model_client,
    )

team = MagenticOneGroupChat(
    [file_surfer, surfer, hiking_assistant],
      model_client=az_model_client, 
      max_turns=50,)

task = f"""
You have been provided with a set of product files from file_surfer to provide product recommendations for a hiking plan.
Web Surfer will search the internet for the weather forecast for the hiking location.
Hiking Assistant will provide a hiking plan based on the location given to you.
You will provide a hiking plan, a weather update and product recommendations for the hike.
You will ask the User Proxy for approval to terminate the conversation.

I want to hike in the Grand Canyon

"""
response = await Console(team.run_stream(task=task))
get_agent_responses(response)


In [ ]:
get_agent_responses(response)